In [1]:
import requests
from copy import copy


def dfs(category_id):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:

        d_id = d['id']
        d_name = d['name']
        d_is_leaf = d['is_leaf']
        
        if(d_is_leaf):
            leaf_id = copy(d_id)
            leaf_name = copy(d_name)
            leaf_category.append([leaf_id,leaf_name])
            
        else:
            dfs(d_id)
    
paths = []
leaf_category = []
starting_category = [1,2,3,4,5,8,9] #출산/육아(6) , 식품 제외(7)


first_category = [1,2,3,4,5]

for category in first_category:
    dfs(category)
#dfs(1)


print(len(leaf_category))
#1 : 패션의류 --> 68개 
#2 : 패션잡화 --> 264개
#3 : 화장품/미용 --> 157개
#4 : 디지털/가전 --> 681개
#5 : 가구/인테리어 --> 354개
#6 : 출산/육아 (X)
#7 : 식품 (X)
#8 : 스포츠/레저 --> 카테고리 : 480개, 키워드 : 154207개 --> 필터 적용 : 20000개
#9 : 생활/건강 --> 카테고리 : 1227개, 키워드 : 465817개 --> 필터 적용 : 61751개

#app 1개당 기본 api 12500개 
#계정 1개당 app 10개, api 125000개

1524


In [2]:
import json
import urllib.request
from tqdm import notebook
from joblib import Parallel, delayed
from time import sleep
from config import *
from infsell import *



def extract_keywords(category,client_auth):
    print(category)
    
    category_id = category[0]
    category_name = category[1]
    
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(category_id)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(category_id)
    response_data = requests.post(url_data, json=PAYLOAD)
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data']
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords,dict_clicks = get_keywords(data)
    keywords_nb = remove_brands(keywords,brands)
    
    for keyword in notebook.tqdm(keywords_nb, desc = "키워드 진행도"):
        encText = urllib.parse.quote(keyword)
        url_excluding_oversea = "https://openapi.naver.com/v1/search/shop.json?display=80&exclude=cbshop&query=" + encText
        url_items = "https://openapi.naver.com/v1/search/shop.json?display=80&query=" + encText # JSON 결과
        
        try:
            items_excluding_oversea = get_items(url_excluding_oversea,client_auth)
            items = get_items(url_items,client_auth)
            dict_excluding_oversea = {}
            
            for item in items_excluding_oversea:
                dict_excluding_oversea[item['productId']]=1

            cnt = 0 
            for item in items :
                if item['productId'] not in dict_excluding_oversea:
                    cnt+=1

            info_keyword={}
            info_keyword['카테고리'] = category_name
            info_keyword['키워드'] = keyword
            info_keyword['검색수'] = dict_clicks[keyword]
            info_keyword['해외배송개수'] = cnt

            final_keywords.append(info_keyword)

        except:
            print('error 발생, 10초 대기 --> 키워드 : ', keyword)
            pass
            
       
        

#멀티코어 개수
num_core = 12

cnt_keyword=0
path_auth='auth_api.txt'
client_auth = get_auth_api(path_auth)
final_keywords = []

with Parallel(n_jobs=num_core, require='sharedmem') as parallel:
    parallel(delayed(extract_keywords)(category,client_auth)for category in notebook.tqdm(leaf_category))
    
print(final_keywords)

  0%|          | 0/1524 [00:00<?, ?it/s]

[660, '코디세트'][683, '티셔츠']
[684, '블라우스/셔츠']

[685, '니트/스웨터']
[686, '카디건']
[687, '원피스']
[688, '스커트']
[689, '청바지']
[690, '바지']
[691, '점프슈트']
[692, '레깅스']
[693, '코트']


키워드 진행도:   0%|          | 0/137 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/241 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/233 [00:00<?, ?it/s]

...

[3568, '쿠션/방석커버세트']


키워드 진행도:   0%|          | 0/44 [00:00<?, ?it/s]

[3569, '쿠션/방석기타']


키워드 진행도:   0%|          | 0/85 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
len(final_keywords)

82754

In [4]:
from infsell import convert_dict_to_excel
import openpyxl

name_file = '2023_06_10_top500_키워드'

convert_dict_to_excel(name_file, final_keywords)

엑셀 변환 완료
